# Manejo de datos scrapeados de canciones de hip hop

Estudio de los datos scrapeados de letras de canciones Hip Hop [https://www.hhgroups.com/](https://www.hhgroups.com/)

Cargo los datos de un archivo pickle previamente descargado por un script de scrapping en Python.

In [1]:
import pickle
import numpy as np
import re

with open('hiphop.txt','r', encoding='utf8') as f:
    rap_file= f.read()

len(rap_file)



26079611

Limpio el artista, en el texto el cantante aparece de forma '[nombre]', para eso creo una función:

In [2]:
def cleanCorchete(text):
    return re.sub(r'(\[.*?\])', '', text, flags=re.DOTALL)

rap_file = cleanCorchete(rap_file)

len(rap_file)

25562157

## Pretratamiento con spacy

Descargo el lenguaje español de spacy para cnfigurarlo, ya que la mayoría del texto que voy a tratar lo utiliza

In [3]:
#!python -m spacy download es_core_news_sm

✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


Limpio el archivo con spacy y lo guardo en un txt porque este proceso lleva tiempo.

In [4]:
#trabajo directamente con el lenguaje español de spacy
import es_core_news_sm

nlp = es_core_news_sm.load()

#regex para limpiar los artistas entre corchetes
rap_file=re.sub(r'(\[.*?\])', '', rap_file, flags=re.DOTALL)
n =0
start =0
aux_clean =''

for i in range(1,27):
  n+=1
  end=1000000 * i
  doc = nlp(rap_file[start:end])
  start =1000000 * i
  filter=['-','.',',','!','?']
  
  for token in doc:

    if token.is_alpha or token.is_digit or (token.is_punct and token.text in filter):

      aux_clean+=token.text

    if token.whitespace_:  

      aux_clean+=token.whitespace_

rap_file=aux_clean
print("número de batchs", n)

with open ('hiphop_cleanv2.txt','w') as fin:
  fin.write(rap_file)

número de batchs 26


# Tratamiento de texto con tensor flow y keras

## Tokenización

Voy a tokenizar el texto mediante los caracteres usando keras.
Primero cargo el fichero para crear el objeto Tokenizer

In [5]:
with open ('hiphop_cleanv2.txt','r') as fout:
  rap_file = fout.read()

print("N caracteres archivo limpio:",len(rap_file))

N caracteres archivo limpio: 24844054


In [6]:
import tensorflow as tf 
from tensorflow import keras

#reduzco el texto para acelerar el entrenamiento
text =rap_file[:1200000]

#creo el toekenizer a nivel caracter y añado uno por defecto para posibles caracteres que no esten en el vocabulario (Out of vocabulary <OOV>)
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,oov_token='<OOV>')
tokenizer.fit_on_texts(text)

#hago una prueba de codificación y decodificación
word_vector =tokenizer.texts_to_sequences(['Barrio'])
print(word_vector)
vector_word=tokenizer.sequences_to_texts(word_vector)
print(vector_word)

#número de caracteres diferentes de mi vocabulario
vocab_size = len(tokenizer.word_index)

#número total de caracteres
nChars = tokenizer.document_count

print('Tamaño del vocabulario:',vocab_size, '- Tamaño del texto:', nChars)


[[21, 4, 8, 8, 9, 5]]
['b a r r i o']
Tamaño del vocabulario: 67 - Tamaño del texto: 1200000


## Dividir el conjunto de datos secuencialmente (Train y Test)

El conjunto de entrenamiento sera el 90%, 10% para test. Para alamacenar estos conjuntos de datos utilizaré el objeto Dataset de tensorflow, con vistas a manejar tensores para posteriormente pasarlo a la red neuronal.

In [10]:
[codificacion] =np.array(tokenizer.texts_to_sequences([text]))-1 # la codificación irá de 0 a 78
#con // se fuerza que el resultado sea un entero
train_size =nChars * 90 //100

train_set = tf.data.Dataset.from_tensor_slices(codificacion[:train_size])
test_set = tf.data.Dataset.from_tensor_slices(codificacion[train_size:])
len(train_set),len(test_set)

(1080000, 120000)

train_set y test_set son dos vectores con un único elemento (todo el texto con los caracteres codificaods a números), para pasarlo a una red neuronal necesito dividirlo en pequeñas porciones de texto, de  101 caracteres. El método window permite realizar esto. Comenzará en a crear ventanas de 100 elementos desde la posición uno, pasando a la dos, tres..., creando un conjuto de vectores de 101 elementos. De esta manera se divide el texto en un conjuto de ventanas de 101 caracteres.

La ventana se configura con el tramaño, shift es el número de pasos que avanza la ventana cada vez y drop_remainder a True hará que el tamaño de la venatana no vaya disminuyendo al final, es decir, para que cuando queden los 101 últimos caracteres codificados el proceso se detenga.

In [11]:
window_size = 101

train_set=train_set.repeat().window(window_size,shift=1,drop_remainder=True)

test_set=test_set.repeat().window(window_size,shift=1,drop_remainder=True)

Aplanamos los conjuntos de datos con el tamaño de la ventana (101), Al tener un conjunto de datos anidados, necesito aplanar esto para conseguir un conjunto de tensores con una longitud fija de 101, que se consigue fácilmente llamando a la función flat_map.

In [12]:
train_set =train_set.flat_map(lambda window : window.batch(window_size))

test_set =test_set.flat_map(lambda window : window.batch(window_size))

Se realiza un mezclado de las ventanas

In [13]:
batch_size = 32

train_set = train_set.shuffle(10000).batch(batch_size)
train_set = train_set.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

test_set = test_set.shuffle(10000).batch(batch_size)
test_set = test_set.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

Voy a realizar la codificación one-hot para crear la bolsa de palabras con los 79 caracteres distintos que se manejaban, divido en una tupla, las secuencia que sirve para predecir y su predicción, y añado la precarga

In [14]:
train_set = train_set.map(lambda X, y:(tf.one_hot(X, depth=vocab_size),y))
train_set=train_set.prefetch(1)

test_set = test_set.map(lambda X, y:(tf.one_hot(X, depth=vocab_size),y))
test_set=test_set.prefetch(1)

## Creación de la red neuronal

In [15]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, vocab_size],
                     #dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     #dropout=0.2, recurrent_dropout=0.2),
                     dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(vocab_size,
                                                    activation="softmax"))
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam",metrics=['accuracy'])



Creo un par de funciones callbacks para el entrenamiento, una paraguardar el modelo por epoch y otra para parar en el caso de que no haya mejora

In [16]:
checkpoint_cb= keras.callbacks.ModelCheckpoint("hiphop_char_model_2/callback_model.h5")
stop_early = keras.callbacks.EarlyStopping(patience=3)

Entreno

In [ ]:
history = model.fit(train_set,
                    epochs=1,
                    validation_data = test_set,
                    callbacks =[checkpoint_cb, stop_early])

In [23]:
import tensorflow as tf 
from tensorflow import keras

from tensorflow.keras.models import model_from_json

checkpoint_cb= keras.callbacks.ModelCheckpoint("hiphop_char_model_2/callback_model.h5")

with open('hiphop_char_model/model.json') as json_file:
    json_config = json_file.read()

model = model_from_json(json_config)
model.load_weights('hiphop_char_model/model.h5')



 191674/Unknown - 62286s 325ms/step - loss: 1.6436 - accuracy: 0.4942

KeyboardInterrupt: 

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam",metrics=['accuracy'])
history = model.fit(train_set,
                    epochs=1, callbacks =[checkpoint_cb])

## Preprocesamiento entrada

Para probar el modelo voy a crear unas funciones auxiliares que realicen el posprocesamiento: la tokenización y la codificación one-hot.
Creo una función para predecir el siguiente caracter y otra para que cree un bucle y genere texto

In [110]:

def treament(input_list):
    input_token =np.array(tokenizer.texts_to_sequences(input_list))-1
    return tf.one_hot(input_token,nDstinctChar)

def next_char(input):
    aux = treament([input])
    predic = np.argmax(model(aux),axis=-1)
    #predic = model(input)[0,-1:,:].numpy() +1
    return tokenizer.sequences_to_texts(predic+1)[0][-1]



def complete_text(text, n_chars=80, temperature=1):
    for i in range(n_chars):
        text += next_char(text)
    return text

input = 'support vector'
complete_text(input)

'support vector machines (e.g. the coefficients \\(\\ell_2\\) is the coefficients \\(\\ell_2\\) is th'

Parece que el modelo, a la hora de predecir, se queda enganchado y repite la misma frase una y otra vez, es más pong lo que pongo siempre converge en el mismo discurso. Necesita introducir algo de aletoriedad

In [113]:
def next_char(input):
    aux = treament([input])
    char_prob = model.predict(aux)[0, -1:,:]
    rescaled_prob = tf.math.log(char_prob) / 0.8
    char_categ = tf.random.categorical(rescaled_prob,num_samples=1)
    return tokenizer.sequences_to_texts(char_categ.numpy() +1 )[0]

input = 'Linear model'
complete_text(input)

'Linear model that computes the above similarity is the\ncoefficients \\(\\ell_0\\) is the model '

## Guardar el modelo de generciónde texto por caracteres

In [117]:
# serializar el modelo a JSON
model_json = model.to_json()
with open("char_model/model.json", "w") as json_file:
    json_file.write(model_json)
# serializar los pesos a HDF5
model.save_weights("char_model/model.h5")


Mejorando el modelo anterior a través del estado

In [11]:

#Creo el dataset de train con el 90%
train_ds_estado = tf.data.Dataset.from_tensor_slices(codificacion[:nChars * 90 // 100])
ds_size=len(train_ds_estado)
#creo las ventanas, esta vez no hay ni solapamiento, el númreo de ventanas se reduce
#tampoco hay mezcla, ya que las ventanas tienen que ser secuenciales donde acaba 1 empieza otra
train_ds_estado = train_ds_estado.window(window_size,shift =100, drop_remainder=True)
train_ds_estado=train_ds_estado.flat_map(lambda w: w.batch(window_size))
#los lotes serán equivalentes a la ventana, de esta manera se respetará la secuencialida entre lotes
train_ds_estado =train_ds_estado.batch(1)
train_ds_estado = train_ds_estado.map(lambda w: (w[:,:-1],w[:,1:]))
train_ds_estado =train_ds_estado.map(lambda X,y: (tf.one_hot(X, depth=nDstinctChar),y))
train_ds_estado=train_ds_estado.prefetch(1)
ds_size

495091

In [14]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True, dropout=0.2, batch_input_shape= [1, None,nDstinctChar]),
    keras.layers.GRU(128, return_sequences=True, stateful=True, dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(nDstinctChar, activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam",metrics=['accuracy'])


In [16]:
steps_per_epoch = ds_size // 100
history = model.fit(train_ds_estado, epochs=50,
                    callbacks=[ResetStatesCallback()])

Epoch 1/50
4950/4950 [==============================] - 241s 49ms/step - loss: 1.4999 - accuracy: 0.5570
Epoch 2/50
4950/4950 [==============================] - 236s 48ms/step - loss: 1.4988 - accuracy: 0.5576
Epoch 3/50
4950/4950 [==============================] - 233s 47ms/step - loss: 1.4989 - accuracy: 0.5578
Epoch 4/50
4950/4950 [==============================] - 234s 47ms/step - loss: 1.4967 - accuracy: 0.5582
Epoch 5/50
4950/4950 [==============================] - 234s 47ms/step - loss: 1.4967 - accuracy: 0.5581
Epoch 6/50
4950/4950 [==============================] - 233s 47ms/step - loss: 1.4977 - accuracy: 0.5577
Epoch 7/50
4950/4950 [==============================] - 234s 47ms/step - loss: 1.4932 - accuracy: 0.5595
Epoch 8/50
4950/4950 [==============================] - 234s 47ms/step - loss: 1.4927 - accuracy: 0.5590
Epoch 9/50
4950/4950 [==============================] - 236s 48ms/step - loss: 1.4929 - accuracy: 0.5592
Epoch 10/50
4950/4950 [==============================] 

El poblema del modelo anterior es que sólo permite hacer predicciones para lotes del mismo tamaño que el entrenado, por lo que la entrada debes de ser de 100 carácteres. Para poder hacer que la entrada tenga un tamaño sin determinar hay que crear una red neuranoal sin estado igual, y copiar los pesos del anterior modelo con estado entrenado a este nuevo modelo

In [13]:
base_model_statless = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, nDstinctChar]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(nDstinctChar,activation="softmax"))
])

Para preparar el anterior modelo para que puedad guardar los pesos del modelo con estado habrá que especificar la estructura del tensor utilizado antes, que era la siguiente (\[1, None,nDstinctChar\]). Para permitir una entrada con cualquier tamaño:

In [160]:
base_model_statless.build(tf.TensorShape([None,None,nDstinctChar]))
base_model_statless.set_weights(model.get_weights())

In [ ]:
aprovecahndo el código anterior

In [163]:
def treament(input_list):
    input_token =np.array(tokenizer.texts_to_sequences(input_list))-1
    return tf.one_hot(input_token,nDstinctChar)

def next_char(input,model):
    aux = treament([input])
    char_prob = model.predict(aux)[0, -1:,:]
    rescaled_prob = tf.math.log(char_prob) / 1
    char_categ = tf.random.categorical(rescaled_prob,num_samples=1)
    return tokenizer.sequences_to_texts(char_categ.numpy() +1 )[0]

def complete_text(text, model,n_chars=80, temperature=1):
    for i in range(n_chars):
        text += next_char(text,model)
    return text

input = 'line'
complete_text(input, base_model_statless)

'support vectors, results. with dimensionality requires a list. the cost of dorical feature ind'

In [17]:
# serializar el modelo a JSON
model_json = model.to_json()
with open("char_model_state/model.json", "w") as json_file:
    json_file.write(model_json)
# serializar los pesos a HDF5
model.save_weights("char_model_state/model.h5")